BIbliotecas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import set_option
from scipy import stats

Datasets

In [2]:
url1='https://github.com/gustavowillam/ML/blob/main/trabalhos/Trab2/train.csv?raw=true'
url2='https://github.com/gustavowillam/ML/blob/main/trabalhos/Trab2/test.csv?raw=true'

In [3]:
train = pd.read_csv(url1, sep=",")
test = pd.read_csv(url2)

In [4]:
set_option('display.max_rows', None)


In [ ]:
print(test)


Analise EDA

In [ ]:
train.info()

In [ ]:
train.shape

excluindo colunas

In [5]:
train = train.drop("PassengerId", axis=1)

In [6]:
train = train.drop("Name", axis=1)

mostrando respostas

In [ ]:
print(train["Transported"].value_counts())
print(train["Transported"].value_counts(normalize=True).map("{:.1%}".format))

#plot the pie chart of education categories
train.Transported.value_counts(normalize=False).plot.pie()
plt.show()

substituindo valores nulos

In [ ]:
train.isnull().sum()

HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

In [7]:
#função que preenche dados ausentes categóricos com a moda

def preencher_com_moda(df, colunas):
    for coluna in colunas:
        moda = stats.mode(df[coluna])[0][0]
        df[coluna].fillna(moda, inplace=True)

In [8]:
#função que preenche dados ausentes com a média


def preencher_com_media(df, colunas):
    for coluna in colunas:
        media = df[coluna].mean()
        df[coluna].fillna(media, inplace=True)


In [9]:
#chamando colunas categoricas do df para preencher valores nulos
dtypes = train.dtypes.to_dict()

for col_name, typ in dtypes.items():
    if (typ == 'object'):
        preencher_com_moda(train,[col_name])
    else:
        preencher_com_media(train,[col_name])

<ipython-input-7-147038306be2>:5: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  moda = stats.mode(df[coluna])[0][0]
<ipython-input-7-147038306be2>:5: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  moda = stats.mode(df[coluna])[0][0]


In [ ]:
train.Cabin.mode()


0    G/734/S
Name: Cabin, dtype: object

In [10]:
train.Cabin.fillna(" G/734/S",inplace=True)

convertendo dados categóricos em numéricos

In [11]:
train = pd.get_dummies(train, columns=["HomePlanet","Destination"])

In [12]:
dtypes = train.dtypes.to_dict()

for col_name, typ in dtypes.items():
    if (typ == 'bool'):
        print(col_name)

CryoSleep
VIP
Transported


In [13]:
train.Cabin = train.Cabin.astype('category')
train.Cabin = train.Cabin.cat.codes

In [14]:
train.CryoSleep = train.CryoSleep.astype('category')
train.CryoSleep = train.CryoSleep.cat.codes

train.Transported = train.Transported.astype('category')
train.Transported = train.Transported.cat.codes

train.VIP = train.VIP.astype('category')
train.VIP = train.VIP.cat.codes

Analisando correlação

In [ ]:
train.corr()['Transported']

Separando os dados

In [15]:
x_train = train.copy()
x_train  = x_train.drop(['Transported'], axis=1) #remove a variável dependente do X
y_train = train[['Transported']]

importando o modelo

In [16]:
from sklearn.linear_model import LogisticRegression

fazendo o modelo

In [17]:
#cria uma instancia do modelo
clf = LogisticRegression()

#ajustando o modelo
clf.fit(x_train, y_train)

y_pred_train = clf.predict(x_train)

#To retrieve the accuracy da classificação:
accuracy_train = clf.score(x_train, y_train)  #mede o score (acuracy) do modelo
print('Acuracia do Treino:', accuracy_train)

Acuracia do Treino: 0.7864948809386862


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


testando o modelo

In [18]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [19]:
test = test.drop("PassengerId", axis=1)

In [20]:
test = test.drop("Name", axis=1)

In [28]:
test.isnull().sum()

HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64

In [ ]:
dtypes = test.dtypes.to_dict()

for col_name, typ in dtypes.items():
    if (typ == 'object'):
        preencher_com_moda(test,[col_name])
    else:
        preencher_com_media(test,[col_name])

In [27]:
test.Cabin.fillna(" G/734/S",inplace=True)

In [29]:
test = pd.get_dummies(test, columns=["HomePlanet","Destination"])

In [30]:
test.Cabin = test.Cabin.astype('category')
test.Cabin = test.Cabin.cat.codes

In [33]:
test.CryoSleep = test.CryoSleep.astype('category')
test.CryoSleep = test.CryoSleep.cat.codes


test.VIP = test.VIP.astype('category')
test.VIP = test.VIP.cat.codes

In [35]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CryoSleep                  4277 non-null   int8   
 1   Cabin                      4277 non-null   int16  
 2   Age                        4277 non-null   float64
 3   VIP                        4277 non-null   int8   
 4   RoomService                4277 non-null   float64
 5   FoodCourt                  4277 non-null   float64
 6   ShoppingMall               4277 non-null   float64
 7   Spa                        4277 non-null   float64
 8   VRDeck                     4277 non-null   float64
 9   HomePlanet_Earth           4277 non-null   uint8  
 10  HomePlanet_Europa          4277 non-null   uint8  
 11  HomePlanet_Mars            4277 non-null   uint8  
 12  Destination_55 Cancri e    4277 non-null   uint8  
 13  Destination_PSO J318.5-22  4277 non-null   uint8

In [36]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CryoSleep                  8693 non-null   int8   
 1   Cabin                      8693 non-null   int16  
 2   Age                        8693 non-null   float64
 3   VIP                        8693 non-null   int8   
 4   RoomService                8693 non-null   float64
 5   FoodCourt                  8693 non-null   float64
 6   ShoppingMall               8693 non-null   float64
 7   Spa                        8693 non-null   float64
 8   VRDeck                     8693 non-null   float64
 9   HomePlanet_Earth           8693 non-null   uint8  
 10  HomePlanet_Europa          8693 non-null   uint8  
 11  HomePlanet_Mars            8693 non-null   uint8  
 12  Destination_55 Cancri e    8693 non-null   uint8  
 13  Destination_PSO J318.5-22  8693 non-null   uint8

In [38]:
y_pred_test = clf.predict(test)


Gerando arquivo para submissão

In [51]:
test_submit = pd.DataFrame()

In [52]:
modelo = pd.read_csv(url2)

In [ ]:
modelo.info()

In [55]:
modelo.isnull().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [57]:


test_submit["PassengerId"] = modelo['PassengerId']
test_submit["Transported"] = y_pred_test

In [60]:
test_submit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  4277 non-null   object
 1   Transported  4277 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 37.7+ KB


In [59]:
test_submit['Transported'] = test_submit.Transported.astype(bool)


In [61]:
test_submit.head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [62]:
test_submit = test_submit.to_csv('submission_feliphe.csv', index=False)